# Reference

**Feature Extraction and Statistical Analysis**

Following the same approach used in the affective touch protocol, we extracted the features as summarized in table: 

1. Npeak: Number of significant SMNA peaks wrw
2. AUC: Area under curve of reconstructed phasic signal wrw (mus s)
3. Peak: Maximum amplitude of significant peaks of SMNA signal wrw 
4. Mean Tonic: Mean value of the tonic component within each image time window 


Likewise, the affective touch, a statiscal analysis among the 5 smells were performed on the arousal and valence scores, and on the EDA phasic and tonic features. 

We distinguished the _event-related phasic analysis_ i.e. EDA was studied within a time window of 5s correspondent to the affective stimulus session, and the _non specific fluctuation and tonic analysis_ comprising the tonic level comparison and the differential value of the tonic features between the post- and pre stimulus session.

the difference among the smeels were studied using the **Friedman test**, and in case of rejection of the **Friedman test** null hypothesis, a post-hoc analysis was performaed by means of a **Bonferroni corrected Wilcoxon signed-rank**.

# Import data 

In [1]:
import numpy as np 
# import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cvxEDA
import datetime
from scipy import stats
from mpl_toolkits import mplot3d
import pylab as pl
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from scipy.stats import ks_2samp
from scipy.stats import ttest_1samp, wilcoxon, ttest_ind, mannwhitneyu
from sklearn.preprocessing import normalize
sns.set(color_codes = True)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
# Sampling frequency 4Hz ~= 0.25s
fl_eda_7 = pd.read_csv('data/exp_7/EDA.csv' , header= None)
fl_eda_7.columns  = ['eda']
fl_eda_freq_7 = fl_eda_7.iloc[1].values
print(len(fl_eda_7))
print('freq: ', fl_eda_freq_7)
fl_eda_7.head(10)

8024
freq:  [4.]


,eda
0,1.528332e+09
1,4.000000e+00
2,0.000000e+00
3,3.623110e-01
4,4.186420e-01
5,4.224830e-01
6,4.199230e-01
7,4.173620e-01
8,4.160820e-01
9,4.135210e-01


In [3]:
# Sampling frequency 4Hz ~= 0.25s
fl_temp_7 = pd.read_csv('data/exp_7/TEMP.csv' , header= None)
fl_temp_7.columns  = ['temp']
fl_temp_freq_7 = fl_temp_7.iloc[1].values
print(len(fl_temp_7))
print('freq: ', fl_temp_freq_7)
fl_temp_7.head(10)

8026
freq:  [4.]


,temp
0,1.528332e+09
1,4.000000e+00
2,3.103000e+01
3,3.103000e+01
4,3.103000e+01
5,3.103000e+01
6,3.103000e+01
7,3.103000e+01
8,3.103000e+01
9,3.103000e+01


In [4]:
fl_init_time_7 = datetime.datetime.utcfromtimestamp(fl_eda_7.iloc[0]).strftime('%Y-%m-%d %H:%M:%S.000')
fl_init_time_7

'2018-06-07 00:48:15.000'

In [5]:
fl_eda_7 = fl_eda_7.iloc[3:]
fl_temp_7 = fl_temp_7.iloc[3:]

In [6]:
#create index 
fl_time_eda_7= pd.date_range(start=fl_init_time_7, periods=len(fl_eda_7), 
                              freq=str(1/int(fl_eda_freq_7))+'S')
fl_time_eda_7

DatetimeIndex([       '2018-06-07 00:48:15', '2018-06-07 00:48:15.250000',
               '2018-06-07 00:48:15.500000', '2018-06-07 00:48:15.750000',
                      '2018-06-07 00:48:16', '2018-06-07 00:48:16.250000',
               '2018-06-07 00:48:16.500000', '2018-06-07 00:48:16.750000',
                      '2018-06-07 00:48:17', '2018-06-07 00:48:17.250000',
               ...
               '2018-06-07 01:21:37.750000',        '2018-06-07 01:21:38',
               '2018-06-07 01:21:38.250000', '2018-06-07 01:21:38.500000',
               '2018-06-07 01:21:38.750000',        '2018-06-07 01:21:39',
               '2018-06-07 01:21:39.250000', '2018-06-07 01:21:39.500000',
               '2018-06-07 01:21:39.750000',        '2018-06-07 01:21:40'],
              dtype='datetime64[ns]', length=8021, freq='250L')

In [7]:
fl_time_temp_7 = pd.date_range(start=fl_init_time_7, periods=len(fl_temp_7), 
                              freq=str(1/int(fl_temp_freq_7))+'S')
fl_time_temp_7

DatetimeIndex([       '2018-06-07 00:48:15', '2018-06-07 00:48:15.250000',
               '2018-06-07 00:48:15.500000', '2018-06-07 00:48:15.750000',
                      '2018-06-07 00:48:16', '2018-06-07 00:48:16.250000',
               '2018-06-07 00:48:16.500000', '2018-06-07 00:48:16.750000',
                      '2018-06-07 00:48:17', '2018-06-07 00:48:17.250000',
               ...
               '2018-06-07 01:21:38.250000', '2018-06-07 01:21:38.500000',
               '2018-06-07 01:21:38.750000',        '2018-06-07 01:21:39',
               '2018-06-07 01:21:39.250000', '2018-06-07 01:21:39.500000',
               '2018-06-07 01:21:39.750000',        '2018-06-07 01:21:40',
               '2018-06-07 01:21:40.250000', '2018-06-07 01:21:40.500000'],
              dtype='datetime64[ns]', length=8023, freq='250L')

In [8]:
fl_eda_7 = fl_eda_7.set_index(fl_time_eda_7, drop =True)
fl_eda_7 = fl_eda_7[:]
fl_eda_7 = fl_eda_7.dropna()
print(len(fl_eda_7))
fl_eda_7.head()

8021


,eda
2018-06-07 00:48:15.000,0.362311
2018-06-07 00:48:15.250,0.418642
2018-06-07 00:48:15.500,0.422483
2018-06-07 00:48:15.750,0.419923
2018-06-07 00:48:16.000,0.417362


In [9]:
fl_temp_7 = fl_temp_7.set_index(fl_time_temp_7, drop =True)
fl_temp_7 = fl_temp_7[:]
fl_temp_7 = fl_temp_7.dropna()
print(len(fl_temp_7))
fl_temp_7.head()

8023


,temp
2018-06-07 00:48:15.000,31.03
2018-06-07 00:48:15.250,31.03
2018-06-07 00:48:15.500,31.03
2018-06-07 00:48:15.750,31.03
2018-06-07 00:48:16.000,31.03


## Tags - Marked Time Stamps

In [10]:
tags_7 = pd.read_csv('data/exp_7/tags/tags.csv' , header= None)
tags_7.columns  =  ['tagged_time']
tags_7.head()

,tagged_time
0,1.528333e+09
1,1.528333e+09
2,1.528333e+09
3,1.528333e+09
4,1.528333e+09


In [11]:
tags_7.tagged_time = [datetime.datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S.000') for x in tags_7.tagged_time]


In [13]:
tags_7.tagged_time

0    2018-06-07 00:49:35.000
1    2018-06-07 00:54:50.000
2    2018-06-07 00:59:39.000
3    2018-06-07 01:01:06.000
4    2018-06-07 01:01:38.000
5    2018-06-07 01:01:54.000
6    2018-06-07 01:06:26.000
7    2018-06-07 01:06:30.000
8    2018-06-07 01:09:34.000
9    2018-06-07 01:16:35.000
Name: tagged_time, dtype: object

In [16]:
tags_7 = tags_7.drop(6)
tags_7

,tagged_time
0,2018-06-07 00:49:35.000
1,2018-06-07 00:54:50.000
2,2018-06-07 00:59:39.000
3,2018-06-07 01:01:06.000
4,2018-06-07 01:01:38.000
5,2018-06-07 01:01:54.000
7,2018-06-07 01:06:30.000
8,2018-06-07 01:09:34.000
9,2018-06-07 01:16:35.000


# Visualization 

# Feature Extraction 

# Pre-experiment Tonic component 

# Post-experiment Tonic component